Goal: recommend products for customers in each group. Groups are derived from customer segment based on aisle.

Method: 
* Popularity Model with user and item feature
* association analysis

In [1]:
import pandas as pd

In [233]:
pd.set_option('max_colwidth',600)

In [ ]:
prior_data_da = pd.read_csv('/Users/mengruiyin/Desktop/project/prior_data_da.csv')

In [ ]:
aisle_list = pd.read_csv('aisle_list.csv')

## Popularity Model

In [179]:
def user_item(df):
    user_data = df[['user_id','order_dow','order_hour_of_day',
                           'add_to_cart_order']]
    item_data = df[['product_name', 'department', 'aisle', 'reordered']]
    return user_data,item_data

In [191]:
product_count = prior_data_da.groupby(['user_id', 'product_name'], as_index = False)['order_id'].count()

In [192]:
product_count.rename(columns = {'order_id':'count'}, inplace = True)

In [196]:
pop_data = prior_data_da.merge(product_count, on = ['user_id', 'product_name']).drop_duplicates(['user_id', 'product_name'])

In [205]:
pop_data

,Unnamed: 0,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle,count
0,0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,beverages,soft drinks,10
10,1,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,dairy eggs,soy lactosefree,1
11,2,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,snacks,popcorn jerky,10
21,3,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,snacks,popcorn jerky,2
23,4,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,household,paper goods,2
25,6,2398795,1,prior,2,3,7,15.0,10258,2,0,Pistachios,117,19,snacks,nuts seeds dried fruit,9
34,8,2398795,1,prior,2,3,7,15.0,13176,4,0,Bag of Organic Bananas,24,4,produce,fresh fruits,2
36,10,2398795,1,prior,2,3,7,15.0,13032,6,0,Cinnamon Toast Crunch,121,14,breakfast,cereal,3
39,14,473747,1,prior,3,3,12,21.0,25133,4,0,Organic String Cheese,21,16,dairy eggs,packaged cheese,8
47,15,473747,1,prior,3,3,12,21.0,30450,5,0,Creamy Almond Butter,88,13,pantry,spreads,1


In [171]:
grouped2 = aisle_list.groupby('topic')
aisle_group = {}
#user_group = {}
for i in range(5):
    aisle_group[i] = grouped2.get_group(i)

In [199]:
cluster = {}
for index in range(5):
    cluster[index] = pop_data[pop_data['user_id'].isin(aisle_group[index].user_id.values)]

In [200]:
aisle0 = graphlab.SFrame(cluster[0])

In [212]:
user_data0, item_data0 = user_item(aisle0)

In [247]:
personalized_model_0 = graphlab.recommender.factorization_recommender.create(aisle0,
                                                                            user_id = 'user_id',
                                                                            item_id = 'product_name',
                                                                            target = 'count',
                                                                            user_data = user_data0,
                                                                            item_data = item_data0
                                                                           )

Recsys training: model = factorization_recommender

Preparing data set.

Data has 2712864 observations with 45067 users and 46409 items.

Data prepared in: 8.50857s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 3.92501

Final training RMSE: 1.98086

In [283]:
certain_str0_0 = ['Soda', 
               'Fridge Pack Cola',
               'Cola',
               'Coke Classic',
               'Ginger Ale']

In [245]:
certain_str0_1 = ['Chocolate Ice Cream', 
               'Vanilla Ice Cream',
               'Chocolate Chip Cookie Dough Ice Cream',
               'Natural Vanilla Ice Cream',
               'Half Baked\xc2\xae Ice Cream']

In [237]:
certain_str0_2 = ['Ultra Soft Facial Tissues', 
              'Select-A-Size White Paper Towels',
               'White Giant Paper Towel Rolls',
               'Select-A-Size Paper Towels, White, 2 Huge Rolls = 5 Regular Rolls  Towels/Napkins',
               'Sensitive Toilet Paper']

In [257]:
certain_str0_3 = ['Beef Franks', 
              'Hardwood SmokedCenter Cut Original Bacon',
               'Uncured Hickory Smoked Sunday Bacon',
               'Hardwood Smoked Bacon',
               'Smok Cured Turkey Bacon']

In [363]:
certain_str0 = [certain_str0_0, certain_str0_1, certain_str0_2, certain_str0_3]

In [364]:
for s in range(len(certain_str0)): 
    appended_data = []
    for item in certain_str0[s]:
        df = personalized_model_0.get_similar_items([item])
        appended_data.append(df)
    personal_recommend0[s] = appended_data   

In [365]:
personal_recommend0[0]

[Columns:
 	product_name	str
 	similar	str
 	score	float
 	rank	int
 
 Rows: 10
 
 Data:
 +--------------+-------------------------------+----------------+------+
 | product_name |            similar            |     score      | rank |
 +--------------+-------------------------------+----------------+------+
 |     Soda     |      Mango Protein Shake      | 0.977502763271 |  1   |
 |     Soda     |  Small Curd Cottage Cheese 4% | 0.954149603844 |  2   |
 |     Soda     |        Breakfast Claws        | 0.949481248856 |  3   |
 |     Soda     | Classic Macaroni and Chees... |  0.9443988204  |  4   |
 |     Soda     | Flashin' Fruit Punch Fruit... | 0.943142235279 |  5   |
 |     Soda     |         Sesame Chicken        | 0.938020944595 |  6   |
 |     Soda     |  Sliced Carrots No Salt Added | 0.934114038944 |  7   |
 |     Soda     |  Lemon Mint Foaming Hand Wash | 0.918240308762 |  8   |
 |     Soda     |   2nd Foods Chicken & Gravy   | 0.910972654819 |  9   |
 |     Soda     |     S

In [374]:
personal_recommend0[0][0].print_rows(max_column_width = 300)

+--------------+----------------------------------+----------------+------+
| product_name |             similar              |     score      | rank |
+--------------+----------------------------------+----------------+------+
|     Soda     |       Mango Protein Shake        | 0.977502763271 |  1   |
|     Soda     |   Small Curd Cottage Cheese 4%   | 0.954149603844 |  2   |
|     Soda     |         Breakfast Claws          | 0.949481248856 |  3   |
|     Soda     | Classic Macaroni and Cheese Cups |  0.9443988204  |  4   |
|     Soda     | Flashin' Fruit Punch Fruit Drink | 0.943142235279 |  5   |
|     Soda     |          Sesame Chicken          | 0.938020944595 |  6   |
|     Soda     |   Sliced Carrots No Salt Added   | 0.934114038944 |  7   |
|     Soda     |   Lemon Mint Foaming Hand Wash   | 0.918240308762 |  8   |
|     Soda     |    2nd Foods Chicken & Gravy     | 0.910972654819 |  9   |
|     Soda     |      Superfood Pesto Sauce       | 0.909017860889 |  10  |
+-----------

In [221]:
aisle1 = graphlab.SFrame(cluster[1])

In [222]:
user_data1, item_data1 = user_item(aisle1)

In [223]:
personalized_model_1 = graphlab.recommender.factorization_recommender.create(aisle1.remove_column('days_since_prior_order'),
                                                                            user_id = 'user_id',
                                                                            item_id = 'product_name',
                                                                            target = 'count',
                                                                            user_data = user_data1,
                                                                            item_data = item_data1
                                                                           )

Recsys training: model = factorization_recommender

Preparing data set.

Data has 1213613 observations with 26159 users and 37835 items.

Data prepared in: 5.51174s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 18.7509

Final training RMSE: 4.33024

In [326]:
certain_str1_1 = ['Milk Chocolate Almonds', 
               'Organic Dark Chocolate Peanut Butter Cups',
               'Almonds & Sea Salt in Dark Chocolate',
               'Almonds',
               'Milk Chocolate Covered Raisins']

In [331]:
certain_str1_2 = ['Unsweetened Premium Iced Tea', 
               'Zen Tea',
               'Premium Unsweetened Iced Tea',
               'Earl Grey Tea',
               'Variety Diet Tea']

In [332]:
certain_str1_3 = ['Unsweetened Almondmilk', 
               'Organic Unsweetened Almond Milk',
               'nsweetened Original Almond Breeze Almond Milk',
               'Unsweetened Vanilla Almond Milk',
               'Vanilla Almond Breeze Almond Milk']

In [333]:
certain_str1 = [certain_str1_1, certain_str1_2, certain_str1_3]

In [375]:
personal_recommend1 = {}
for s in range(len(certain_str1)): 
    appended_data = []
    for item in certain_str1[s]:
        df = personalized_model_1.get_similar_items([item])
        appended_data.append(df)
    personal_recommend1[s] = appended_data   

In [376]:
personal_recommend1[0]

[Columns:
 	product_name	str
 	similar	str
 	score	float
 	rank	int
 
 Rows: 10
 
 Data:
 +------------------------+-------------------------------+----------------+------+
 |      product_name      |            similar            |     score      | rank |
 +------------------------+-------------------------------+----------------+------+
 | Milk Chocolate Almonds | Lunchables Turkey & Americ... | 0.999949336052 |  1   |
 | Milk Chocolate Almonds |          Swiss Rolls          | 0.999941408634 |  2   |
 | Milk Chocolate Almonds | Dark Chocolate Pumpkin See... | 0.999922990799 |  3   |
 | Milk Chocolate Almonds | Pretzel In Milk Chocolate Bar | 0.999904453754 |  4   |
 | Milk Chocolate Almonds | Dark Chocolate With Carame... | 0.999868869781 |  5   |
 | Milk Chocolate Almonds | Rapid Relief Reflux Relief... | 0.999866962433 |  6   |
 | Milk Chocolate Almonds | Apricot  Walnut & Coconut ... | 0.999865233898 |  7   |
 | Milk Chocolate Almonds | Vegetable Chips, Taro, Sea... | 0.999844372

In [361]:
personal_recommend1[1][0].print_rows(max_column_width = 300)

+------------------------------+-----------------------------------------------------+----------------+------+
|         product_name         |                       similar                       |     score      | rank |
+------------------------------+-----------------------------------------------------+----------------+------+
| Unsweetened Premium Iced Tea |                Premium Glacier Water                | 0.999877810478 |  1   |
| Unsweetened Premium Iced Tea | Fragrance Free Natural Confidence Deodorant Crystal | 0.999869406223 |  2   |
| Unsweetened Premium Iced Tea |                 Organic Roots Juice                 | 0.999849438667 |  3   |
| Unsweetened Premium Iced Tea |                 Black Tea With Lemon                | 0.999817311764 |  4   |
| Unsweetened Premium Iced Tea |             Crystal Fresh Drinking Water            | 0.999808847904 |  5   |
| Unsweetened Premium Iced Tea |        Original Sparkling Seltzer Water Cans        | 0.999805033207 |  6   |
|

In [338]:
personal_recommend1[2]

[Columns:
 	product_name	str
 	similar	str
 	score	float
 	rank	int
 
 Rows: 10
 
 Data:
 +------------------------+-------------------------------+----------------+------+
 |      product_name      |            similar            |     score      | rank |
 +------------------------+-------------------------------+----------------+------+
 | Unsweetened Almondmilk |      Original Almondmilk      | 0.999992549419 |  1   |
 | Unsweetened Almondmilk | Organic Pomegranate Lemona... | 0.999989748001 |  2   |
 | Unsweetened Almondmilk | Organic Coconut Lime Kombucha | 0.999985575676 |  3   |
 | Unsweetened Almondmilk |        Mocha Cappuccino       | 0.999984443188 |  4   |
 | Unsweetened Almondmilk |  Chewy Fruit & Veggie Snacks  | 0.999984383583 |  5   |
 | Unsweetened Almondmilk | Lemon Sparkling Natural Mi... | 0.999984145164 |  6   |
 | Unsweetened Almondmilk |   Organic Raw Kombucha Case   | 0.999983847141 |  7   |
 | Unsweetened Almondmilk | Juice Blend, Organic Veget... | 0.999983489

In [224]:
aisle2 = graphlab.SFrame(cluster[2])

In [225]:
user_data2, item_data2 = user_item(aisle2)

In [226]:
personalized_model_2 = graphlab.recommender.factorization_recommender.create(aisle2.remove_column('days_since_prior_order'),
                                                                            user_id = 'user_id',
                                                                            item_id = 'product_name',
                                                                            target = 'count',
                                                                            user_data = user_data2,
                                                                            item_data = item_data2
                                                                           )

Recsys training: model = factorization_recommender

Preparing data set.

Data has 1613589 observations with 34245 users and 36646 items.

Data prepared in: 5.01518s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 16.7802

Final training RMSE: 4.09636

In [339]:
certain_str2_0 = ['Total 2% with Strawberry Lowfat Greek Strained Yogurt', 
               'Total 2% Lowfat Greek Strained Yogurt with Peach',
               'Total 2% Lowfat Greek Strained Yogurt With Blueberry',
               'Total 2% All Natural Greek Strained Yogurt with Honey',
               'Total 2% Greek Strained Yogurt with Cherry 5.3 oz']

In [377]:
appended_data = []
for item in certain_str2_0:
    df = personalized_model_1.get_similar_items([item])
    appended_data.append(df)

In [378]:
appended_data

[Columns:
 	product_name	str
 	similar	str
 	score	float
 	rank	int
 
 Rows: 10
 
 Data:
 +-------------------------------+-------------------------------+----------------+
 |          product_name         |            similar            |     score      |
 +-------------------------------+-------------------------------+----------------+
 | Total 2% with Strawberry L... | Superfood Premium Fruit Sm... | 0.99999588728  |
 | Total 2% with Strawberry L... | Medium Roast Extra Bold K-... | 0.999995291233 |
 | Total 2% with Strawberry L... | French Roast Single Serve ... | 0.999994337559 |
 | Total 2% with Strawberry L... |   Shade Grown Ground Coffee   | 0.999990999699 |
 | Total 2% with Strawberry L... |       Organic Lime Juice      | 0.999990820885 |
 | Total 2% with Strawberry L... | Medium Roast Extra Bold De... | 0.999990522861 |
 | Total 2% with Strawberry L... |       Owl's Howl Coffee       | 0.999990224838 |
 | Total 2% with Strawberry L... | Organic Strawberry Super C... | 0.99

In [380]:
appended_data[0].print_rows(max_column_width = 80)

+-------------------------------------------------------+
|                      product_name                     |
+-------------------------------------------------------+
| Total 2% with Strawberry Lowfat Greek Strained Yogurt |
| Total 2% with Strawberry Lowfat Greek Strained Yogurt |
| Total 2% with Strawberry Lowfat Greek Strained Yogurt |
| Total 2% with Strawberry Lowfat Greek Strained Yogurt |
| Total 2% with Strawberry Lowfat Greek Strained Yogurt |
| Total 2% with Strawberry Lowfat Greek Strained Yogurt |
| Total 2% with Strawberry Lowfat Greek Strained Yogurt |
| Total 2% with Strawberry Lowfat Greek Strained Yogurt |
| Total 2% with Strawberry Lowfat Greek Strained Yogurt |
| Total 2% with Strawberry Lowfat Greek Strained Yogurt |
+-------------------------------------------------------+
+----------------------------------------------------------+----------------+------+
|                         similar                          |     score      | rank |
+-----------------

Recommendations for yogurt for this group of customers are tea, coffee, smoothies, cocktail……most of them of drinks, which makes sense. From recommendation based on factorization personality model, customers in this group like yougurt also like coffee. Customers buying yougurt with low fat also like products with low calorie, such as G2 Low Calorie Fruit Punch Drink. Also customers buying strawberry yougurt also like ohter products with fruits flavor such as Superfood Premium Fruit Smoothie Blend Blueberry.          

In [217]:
aisle3 = graphlab.SFrame(cluster[3])

In [218]:
user_data3, item_data3 = user_item(aisle3)

In [219]:
personalized_model_3 = graphlab.recommender.factorization_recommender.create(aisle3.remove_column('days_since_prior_order'),
                                                                            user_id = 'user_id',
                                                                            item_id = 'product_name',
                                                                            target = 'count',
                                                                            user_data = user_data3,
                                                                            item_data = item_data3
                                                                           )

Recsys training: model = factorization_recommender

Preparing data set.

Data has 1660212 observations with 20496 users and 37768 items.

Data prepared in: 7.98707s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 14.1698

Final training RMSE: 3.76428

In [227]:
aisle4 = graphlab.SFrame(cluster[4])

In [228]:
user_data4, item_data4 = user_item(aisle4)

In [229]:
personalized_model_4 = graphlab.recommender.factorization_recommender.create(aisle4.remove_column('days_since_prior_order'),
                                                                            user_id = 'user_id',
                                                                            item_id = 'product_name',
                                                                            target = 'count',
                                                                            user_data = user_data4,
                                                                            item_data = item_data4
                                                                           )

Recsys training: model = factorization_recommender

Preparing data set.

Data has 6107675 observations with 80242 users and 44415 items.

Data prepared in: 16.0319s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 10.2611

Final training RMSE: 3.20329

In [424]:
certain_str4_0 = ['Organic Cilantro', 
                'Organic Italian Parsley Bunch',
                'Bunched Cilantro',
                'Organic Basil',
                'Organic Thyme']

In [418]:
certain_str4_1 = ['Organic Unsweetened Almond Milk', 
                'Unsweetened Almondmilk',
                'Organic Coconut Milk',
                'Unsweetened Original Almond Breeze Almond Milk',
                'Vanilla Almond Breeze Almond Milk']

In [425]:
certain_str4 = [certain_str4_0, certain_str4_1]
certain_str4

[['Organic Cilantro',
  'Organic Italian Parsley Bunch',
  'Bunched Cilantro',
  'Organic Basil',
  'Organic Thyme'],
 ['Organic Unsweetened Almond Milk',
  'Unsweetened Almondmilk',
  'Organic Coconut Milk',
  'Unsweetened Original Almond Breeze Almond Milk',
  'Vanilla Almond Breeze Almond Milk']]

In [426]:
personal_recommend4 = {}
for s in range(len(certain_str4)): 
    data = []
    for item in certain_str4[s]:
        df = personalized_model_4.get_similar_items([item])
        data.append(df)
    personal_recommend4[s] = data  

In [428]:
personal_recommend4[1]

[Columns:
 	product_name	str
 	similar	str
 	score	float
 	rank	int
 
 Rows: 10
 
 Data:
 +-------------------------------+-------------------------------+----------------+
 |          product_name         |            similar            |     score      |
 +-------------------------------+-------------------------------+----------------+
 | Organic Unsweetened Almond... |  Organic Reduced Fat 2% Milk  | 0.999997019768 |
 | Organic Unsweetened Almond... | Italian Sparkling Mineral ... | 0.999995946884 |
 | Organic Unsweetened Almond... |       Organic Eggs Large      | 0.999995529652 |
 | Organic Unsweetened Almond... |     Natural Artesian Water    | 0.999995112419 |
 | Organic Unsweetened Almond... |  Organic Original Almond Milk | 0.999994337559 |
 | Organic Unsweetened Almond... |          Organic Milk         | 0.999992966652 |
 | Organic Unsweetened Almond... |           Large Eggs          | 0.999992072582 |
 | Organic Unsweetened Almond... | Cage Free Grade AA Large W... | 0.99

In [359]:
appended_data2[0].print_rows(max_column_width = 300)

+------------------+---------------------------------------------+----------------+------+
|   product_name   |                   similar                   |     score      | rank |
+------------------+---------------------------------------------+----------------+------+
| Organic Cilantro |         Organic Kosher Dill Pickles         | 0.999394476414 |  1   |
| Organic Cilantro |           Outrageous B & B Pickles          | 0.999060750008 |  2   |
| Organic Cilantro |  Organic Fruit Yogurt Smoothie Peach Banana | 0.999053239822 |  3   |
| Organic Cilantro |            Organic Hearts Of Palm           | 0.998860239983 |  4   |
| Organic Cilantro |            Classic Round Crackers           | 0.998631834984 |  5   |
| Organic Cilantro |       100% Recycled Bath Tissue Rolls       | 0.998419106007 |  6   |
| Organic Cilantro |       Sunny Days Strawberry Snack Bars      | 0.998270273209 |  7   |
| Organic Cilantro |              Super Greens Salad             | 0.998220205307 |  8   |

### group0 - recommend for Soda, Ice Cream, Paper Goods and hot dogs bacon sausage

In [19]:
recommend = {}
for item in certain_str:
    print 'recommendation for' + str(item) + ':'
    recommend[item] = df[df['antecedants'].apply(lambda x: str(x)).str.contains(item)]


In [57]:
prior_data_da[prior_data_da['product_name'].str.contains('Apricot')]

,Unnamed: 0,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
1086,1086,2363981,13,prior,6,0,18,7.0,31134,5,0,Mediterranean Apricots,117,19,snacks,nuts seeds dried fruit
2778,2778,3374715,27,prior,40,4,10,3.0,24236,13,0,Organic Dried & Unsulfured Orchard Select Apr...,117,19,snacks,nuts seeds dried fruit
2804,2804,3037479,27,prior,45,4,9,2.0,24236,1,1,Organic Dried & Unsulfured Orchard Select Apr...,117,19,snacks,nuts seeds dried fruit
3948,3948,813240,31,prior,19,0,17,4.0,12962,25,0,Apricot Preserves,88,13,pantry,spreads
4441,4441,423623,36,prior,18,4,17,18.0,29298,8,0,Deluxe Thick Apricot Fruit Spread,88,13,pantry,spreads
7923,7923,2568280,63,prior,2,0,9,11.0,12962,9,0,Apricot Preserves,88,13,pantry,spreads
9195,9195,21889,71,prior,10,2,15,11.0,37973,14,0,Apricot-Raspberry Preserves,88,13,pantry,spreads
9937,9937,1574973,77,prior,4,2,9,16.0,6342,3,0,Santa Cruz OG Apricot Fruit Spread,88,13,pantry,spreads
16723,16723,505689,138,prior,9,6,12,7.0,1000,5,0,Apricots,18,10,bulk,bulk dried fruits vegetables
17014,17014,551518,140,prior,13,4,19,7.0,1000,27,0,Apricots,18,10,bulk,bulk dried fruits vegetables


In [65]:
rules0 = pd.read_csv('rules0.csv', index_col = 0)

In [71]:
certain_str0 = ['Soda', 
               'Fridge Pack Cola',
               'Cola',
               'Coke Classic',
               'Ginger Ale']


In [79]:
certain_str0_1 = ['Chocolate Ice Cream', 
               'Vanilla Ice Cream',
               'Chocolate Chip Cookie Dough Ice Cream',
               'Natural Vanilla Ice Cream',
               'Half Baked\xc2\xae Ice Cream']

In [100]:
certain_str0_2 = ['Ultra Soft Facial Tissues', 
               'Select-A-Size White Paper Towels',
               'White Giant Paper Towel Rolls',
               'Select-A-Size Paper Towels, White, 2 Huge Rolls = 5 Regular Rolls  Towels/Napkins',
               'Sensitive Toilet Paper']

In [109]:
recommend0 = {}
for item in certain_str0:
    #print 'recommendation for' + str(item) + ':'
    recommend0[item] = rules0[rules0['antecedants'].apply(lambda x: str(x)).str.contains(item)].\
                        sort_values('lift', ascending = False)

In [110]:
recommend0[certain_str0[0]]

,antecedants,consequents,support,confidence,lift
18,frozenset(['Pure Baking Soda']),frozenset(['Banana']),0.051542,0.409991,1.444816


Based on Association Rules, there is almost no recommended products and some rules recommend banana for. The recommendation provides little informaiton since bananas are the most popular products. That means generally most of people will buy bananas for one or more times. So whatever the products customers buy, it will recommend bananas. For other products like in ice cream ice, there is no recommendations based on association rules. So we cannot use association rules to recommend products for group 0.

In [444]:
[x.print_rows(max_column_width = 300) for x in personal_recommend0[0]]

+--------------+----------------------------------+----------------+------+
| product_name |             similar              |     score      | rank |
+--------------+----------------------------------+----------------+------+
|     Soda     |       Mango Protein Shake        | 0.977502763271 |  1   |
|     Soda     |   Small Curd Cottage Cheese 4%   | 0.954149603844 |  2   |
|     Soda     |         Breakfast Claws          | 0.949481248856 |  3   |
|     Soda     | Classic Macaroni and Cheese Cups |  0.9443988204  |  4   |
|     Soda     | Flashin' Fruit Punch Fruit Drink | 0.943142235279 |  5   |
|     Soda     |          Sesame Chicken          | 0.938020944595 |  6   |
|     Soda     |   Sliced Carrots No Salt Added   | 0.934114038944 |  7   |
|     Soda     |   Lemon Mint Foaming Hand Wash   | 0.918240308762 |  8   |
|     Soda     |    2nd Foods Chicken & Gravy     | 0.910972654819 |  9   |
|     Soda     |      Superfood Pesto Sauce       | 0.909017860889 |  10  |
+-----------

[None, None, None, None, None]

These are results from personalized model. The target of the model is count of each product for every user. The model includes item features: department and aisle and user features: order day, order hour and add to cart order. Here printed are recommendations for products in soft drinks aisle. We can see the model would recommend mango protein shake, cheese and chicken for soda, recommend ice cream and chocolate for fridge pack cola. Vendors could make bundle salkes based on recommendations or activities such as if customers buy 5 soda, they could get a Mango Protein Shake by free.  

According to analysis we got before, the order for soda is different from others. It has high percentage order number at 10am and low percentage at 16pm. It has the highest order number on Monday and second highest order number on Sunday. But the order number would decrease from Monday to Thursday. So the best time for promotion might be Monday or Sudany around 10 am. 

In [92]:
# recommend0_1 = {}
# for item in certain_str0_1:
#     #print 'recommendation for' + str(item) + ':'
#     recommend0_1[item] = rules0[rules0['antecedants'].apply(lambda x: str(x)).str.contains(item)].\
#                         sort_values('lift', ascending = False)

In [101]:
# recommend0_2 = {}
# for item in certain_str0_2:
#     #print 'recommendation for' + str(item) + ':'
#     recommend0_2[item] = rules0[rules0['antecedants'].apply(lambda x: str(x)).str.contains(item)].\
#                         sort_values('lift', ascending = False)

### group1 - recommendation for sparkling water and tea

In [111]:
rules1 = pd.read_csv('rules1.csv', index_col=0)

In [133]:
certain_str1 = ['Sparkling Water Grapefruit', 
               'Spring Water',
               'Lime Sparkling Water',
               'Sparkling Lemon Water',
               'Sparkling Natural Mineral Water']

In [135]:
certain_str1_1 = ['Milk Chocolate Almonds', 
               'Organic Dark Chocolate Peanut Butter Cups',
               'Almonds & Sea Salt in Dark Chocolate',
               'Almonds',
               'Milk Chocolate Covered Raisins']

In [113]:
recommend1 = {}
for item in certain_str1:
    #print 'recommendation for' + str(item) + ':'
    recommend1[item] = rules1[rules1['antecedants'].apply(lambda x: str(x)).str.contains(item)].\
                        sort_values('lift', ascending = False)

In [130]:
sparkling = recommend1[certain_str1[0]][recommend1[certain_str1[0]].lift >=5] 
sparkling.groupby('antecedants').agg({'consequents':lambda x: tuple(x)})

,consequents
antecedants,
"frozenset(['Banana', 'Sparkling Water Grapefruit'])","(frozenset(['Sparkling Water Berry']), frozenset(['Peach Pear Flavored Sparkling Water']), frozenset(['Sparkling Lemon Water']))"
"frozenset(['Lime Sparkling Water', 'Sparkling Water Grapefruit', 'Banana'])","(frozenset(['Sparkling Lemon Water']),)"
"frozenset(['Lime Sparkling Water', 'Sparkling Water Grapefruit'])","(frozenset(['Peach Pear Flavored Sparkling Water', 'Sparkling Lemon Water']), frozenset(['Sparkling Lemon Water', 'Sparkling Water Berry']), frozenset(['Peach Pear Flavored Sparkling Water', 'Sparkling Water Berry']), frozenset(['Sparkling Water Berry']), frozenset(['Sparkling Lemon Water']), frozenset(['Peach Pear Flavored Sparkling Water']), frozenset(['Pure Sparkling Water']))"
"frozenset(['Organic Strawberries', 'Sparkling Water Grapefruit'])","(frozenset(['Lime Sparkling Water']),)"
"frozenset(['Peach Pear Flavored Sparkling Water', 'Lime Sparkling Water', 'Sparkling Water Grapefruit'])","(frozenset(['Sparkling Water Berry']), frozenset(['Sparkling Lemon Water']))"
"frozenset(['Peach Pear Flavored Sparkling Water', 'Sparkling Lemon Water', 'Sparkling Water Grapefruit'])","(frozenset(['Lime Sparkling Water']),)"
"frozenset(['Peach Pear Flavored Sparkling Water', 'Sparkling Water Berry', 'Sparkling Water Grapefruit'])","(frozenset(['Lime Sparkling Water']),)"
"frozenset(['Peach Pear Flavored Sparkling Water', 'Sparkling Water Grapefruit'])","(frozenset(['Sparkling Water Berry', 'Lime Sparkling Water']), frozenset(['Sparkling Water Berry']), frozenset(['Sparkling Lemon Water', 'Lime Sparkling Water']), frozenset(['Sparkling Lemon Water']), frozenset(['Lime Sparkling Water']))"
"frozenset(['Pure Sparkling Water', 'Sparkling Water Grapefruit'])","(frozenset(['Lime Sparkling Water']),)"


Based on clustering result, the most popular aisle is water seltzer sparkling water, so I used association rules to get recommendation based on top 5 products in this aisle. The model provides multiple rules and here is only a part of screenshot. The model provides many recommendations for products in water seltzer sparkling water aisle. So I select rules with lift values greater than 5. Except getting association rules, we also can find Lime or Lemon flavors are very popular. They are associated with many products in sparkling water aisle. So if there is no information about what products should recommend to the specific product in sparkling water aisle, vendors could recommend lemon or lime flavors. It is a safe choice. Also, vendors could develop new products of lime or lemon flavors for this group of customers. Since sparkling water is the most popular aisle in this group, so association rules could provide valued information(association rules are built based on frequent). However there is no recommendation for tea, and products for other aisles. We could get recommendation for these products by other recommendation methods.


In [389]:
personal_recommend1[1]

[Columns:
 	product_name	str
 	similar	str
 	score	float
 	rank	int
 
 Rows: 10
 
 Data:
 +------------------------------+-------------------------------+----------------+
 |         product_name         |            similar            |     score      |
 +------------------------------+-------------------------------+----------------+
 | Unsweetened Premium Iced Tea |     Premium Glacier Water     | 0.999877810478 |
 | Unsweetened Premium Iced Tea | Fragrance Free Natural Con... | 0.999869406223 |
 | Unsweetened Premium Iced Tea |      Organic Roots Juice      | 0.999849438667 |
 | Unsweetened Premium Iced Tea |      Black Tea With Lemon     | 0.999817311764 |
 | Unsweetened Premium Iced Tea |  Crystal Fresh Drinking Water | 0.999808847904 |
 | Unsweetened Premium Iced Tea | Original Sparkling Seltzer... | 0.999805033207 |
 | Unsweetened Premium Iced Tea | Vitamin Water Zero Squeeze... | 0.999770343304 |
 | Unsweetened Premium Iced Tea |           Smartwater          | 0.99976837635  

To get recommended products for tea aisle, I used personality model. Here is an example of result for recommendation based on product unsweetened premium iced tea, the most popular products in tea aisle. From personality model, we may recommend Premium Glacier Water, Organic Roots Juice, Black Tea with Lemon……We can see most of recommended products are healthy products. Generally customers in group 1 lead healthier life than customers in group 0.

Sparkling water products have the same rule with soda, so basically the best time to make promotion for group 2 customers is the same as the group 1 customers.

### group 2 - recommendation for yogourt

In [141]:
rules2 = pd.read_csv('rules2.csv', index_col=0)

In [142]:
certain_str2 = ['Total 2% with Strawberry Lowfat Greek Strained Yogurt', 
               'Total 2% Lowfat Greek Strained Yogurt with Peach',
               'Total 2% Lowfat Greek Strained Yogurt With Blueberry',
               'Total 2% All Natural Greek Strained Yogurt with Honey',
               'Total 2% Greek Strained Yogurt with Cherry 5.3 oz']

In [143]:
recommend2 = {}
for item in certain_str2:
    #print 'recommendation for' + str(item) + ':'
    recommend2[item] = rules2[rules2['antecedants'].apply(lambda x: str(x)).str.contains(item)].\
                        sort_values('lift', ascending = False)

In [151]:
recommend2[certain_str2[3]]

,antecedants,consequents,support,confidence,lift


In [400]:
print 'recommendation for yogurt:'
appended_data

recommendation for yogurt:


[Columns:
 	product_name	str
 	similar	str
 	score	float
 	rank	int
 
 Rows: 10
 
 Data:
 +-------------------------------+-------------------------------+----------------+
 |          product_name         |            similar            |     score      |
 +-------------------------------+-------------------------------+----------------+
 | Total 2% with Strawberry L... | Superfood Premium Fruit Sm... | 0.99999588728  |
 | Total 2% with Strawberry L... | Medium Roast Extra Bold K-... | 0.999995291233 |
 | Total 2% with Strawberry L... | French Roast Single Serve ... | 0.999994337559 |
 | Total 2% with Strawberry L... |   Shade Grown Ground Coffee   | 0.999990999699 |
 | Total 2% with Strawberry L... |       Organic Lime Juice      | 0.999990820885 |
 | Total 2% with Strawberry L... | Medium Roast Extra Bold De... | 0.999990522861 |
 | Total 2% with Strawberry L... |       Owl's Howl Coffee       | 0.999990224838 |
 | Total 2% with Strawberry L... | Organic Strawberry Super C... | 0.99

I want to recommend products for customers like yogurt. However association rules model does not return any rules to us, so I used personality model. I recommended products based on top 5 products in yogurt aisle: Total 2% with Strawberry Lowfat Greek Strained Yogurt , Total 2% Lowfat Greek Strained Yogurt with Peach,  Total 2% Lowfat Greek Strained Yogurt With Blueberry,  Total 2% All Natural Greek Strained Yogurt with Honey and Total 2% Greek Strained Yogurt with Cherry 5.3 oz. Here is an example of recommendation results for Total 2% with Strawberry Lowfat Greek Strained Yogurt. We can see Recommendations for yogurt for this group of customers are tea, coffee, smoothies, cocktail……most of them of drinks, which makes sense. From recommendations, customers in this group like yougurt also like coffee. Customers buying yogurt with low fat also like products with low calorie, such as G2 Low Calorie Fruit Punch Drink. And customers buying strawberry yogurt also like other products with fruits flavor such as Superfood Premium Fruit Smoothie Blend Blueberry. 


### group 3 - recommendation for baby food 

In [2]:
rules3 = pd.read_csv('rules3.csv', index_col=0)

In [153]:
certain_str3 = ['Baby Food Stage 2 Blueberry Pear & Purple Carrot', 
                   'Spinach Peas & Pear Stage 2 Baby Food',
                   'Broccoli & Apple Stage 2 Baby Food',
                   'Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food',
                   'Gluten Free SpongeBob Spinach Littles']

In [154]:
recommend3 = {}
for item in certain_str3:
    #print 'recommendation for' + str(item) + ':'
    recommend3[item] = rules3[rules3['antecedants'].apply(lambda x: str(x)).str.contains(item)].\
                        sort_values('lift', ascending = False)

In [155]:
print 'recommendation for ' + str(certain_str3[0])
recommend3[certain_str3[0]][recommend3[certain_str3[0]].lift >=5] 

recommendation for Baby Food Stage 2 Blueberry Pear & Purple Carrot


,antecedants,consequents,support,confidence,lift
26975,frozenset(['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),frozenset(['Baby Food Stage 2 Pumpkin Banana']),0.093062,0.322665,6.234441
2343,frozenset(['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),"frozenset(['Peach, Apricot & Banana Stage 2 Baby Food'])",0.093062,0.362539,5.973587
6074,frozenset(['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),frozenset(['Broccoli & Apple Stage 2 Baby Food']),0.093062,0.435992,5.817293
23793,frozenset(['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),frozenset(['Spinach Peas & Pear Stage 2 Baby Food']),0.093062,0.477440,5.668662


In [156]:
print 'recommendation for ' + str(certain_str3[1])
recommend3[certain_str3[1]][recommend3[certain_str3[1]].lift >=5] 

recommendation for Spinach Peas & Pear Stage 2 Baby Food


,antecedants,consequents,support,confidence,lift
22034,frozenset(['Spinach Peas & Pear Stage 2 Baby Food']),frozenset(['Broccoli & Apple Stage 2 Baby Food']),0.084224,0.516522,6.891780
23792,frozenset(['Spinach Peas & Pear Stage 2 Baby Food']),frozenset(['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),0.084224,0.527536,5.668662


Customers buying Spinach Peas & Pear Stage 2 Baby Food also likes Broccoli&Apple Stage 2 Baby Food and Baby Food Stage 2 Blueberry Pear&Purple Carrot.

In [157]:
print 'recommendation for ' + str(certain_str3[2])
recommend3[certain_str3[2]][recommend3[certain_str3[2]].lift >=5]

recommendation for Broccoli & Apple Stage 2 Baby Food


,antecedants,consequents,support,confidence,lift
22035,frozenset(['Broccoli & Apple Stage 2 Baby Food']),frozenset(['Spinach Peas & Pear Stage 2 Baby Food']),0.074948,0.580456,6.891780
6075,frozenset(['Broccoli & Apple Stage 2 Baby Food']),frozenset(['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),0.074948,0.541368,5.817293


In [158]:
print 'recommendation for ' + str(certain_str3[3])
recommend3[certain_str3[3]]

recommendation for Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food


,antecedants,consequents,support,confidence,lift
16541,frozenset(['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food']),frozenset(['Organic Strawberries']),0.052244,0.671963,1.593800
26443,frozenset(['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food']),frozenset(['Banana']),0.052244,0.592523,1.276075


In [159]:
print 'recommendation for ' + str(certain_str3[4])
recommend3[certain_str3[4]]

recommendation for Gluten Free SpongeBob Spinach Littles


,antecedants,consequents,support,confidence,lift
7691,frozenset(['Gluten Free SpongeBob Spinach Littles']),frozenset(['Organic Baby Spinach']),0.060397,0.523848,1.648069
16255,frozenset(['Gluten Free SpongeBob Spinach Littles']),frozenset(['Organic Strawberries']),0.060397,0.689572,1.635566
10918,frozenset(['Gluten Free SpongeBob Spinach Littles']),frozenset(['Bag of Organic Bananas']),0.060397,0.548100,1.480369
26259,frozenset(['Gluten Free SpongeBob Spinach Littles']),frozenset(['Banana']),0.060397,0.561843,1.210001


In [392]:
from IPython.display import Image

In [399]:
img = 'Picture1.png'
Image(url=img, width=700, height=500)

Based on my the association rule, customers buying Baby Food Stage 2 Bluberry Pear & Purple Carrot also would like to buy Baby Food Stage 2 Pumpkin Banana, Peach, Apricot & Banana Stage 2 Baby Food, Broccoli & Apple Stage 2 Baby Food and Spinach Peas & Pear Stage 2 Baby Food.

The model also recommend Baby Food Stage 2 Blueberry pear & Purple Carrot for customers who like Broccoli & Apple stage 2 Baby Food and Spinach Peas & Pear Stage 2 Baby Food. So we can see here are doulbe arrows between these three products.

So based on my results, venders could recommend bundle sales. A bundle cound include one of the above flavors of Baby Food 2 especially could include each of these there products so that customers could buy all flavors they like or most of them are flavors they like instead only one flavor. Aslo, venders could launch sales like for people buying 5 Baby Food Stage 2 Bluberry Pear & Purple Carrot, they could get one of any other flavor's Baby Food Stage 2 listed above for free. The activity could encourage more people to buy more products. 

In [398]:
img = 'Picture2.png'
Image(url=img, width=500, height=300)

Customers like buying Gluten Free SpongeBob Spinach Littles really like spinach. So vendors could recommend Oranic Baby Spinach to these customers. 

### group 4

In [160]:
rules4 = pd.read_csv('rules4.csv', index_col=0)

In [402]:
certain_str4 = ['Organic Unsweetened Almond Milk', 
                'Unsweetened Almondmilk',
                'Organic Coconut Milk',
                'Unsweetened Original Almond Breeze Almond Milk',
                'Vanilla Almond Breeze Almond Milk']

In [162]:
recommend4 = {}
for item in certain_str4:
    #print 'recommendation for' + str(item) + ':'
    recommend4[item] = rules4[rules4['antecedants'].apply(lambda x: str(x)).str.contains(item)].\
                        sort_values('lift', ascending = False)

In [403]:
print "recommendation for soy lactosefree products"
recommend4[certain_str4[0]]

recommendation for soy lactosefree products


,antecedants,consequents,support,confidence,lift
3037,frozenset(['Organic Unsweetened Almond Milk']),frozenset(['Organic Hass Avocado']),0.109602,0.514669,1.586621
10243,frozenset(['Organic Unsweetened Almond Milk']),frozenset(['Bag of Organic Bananas']),0.109602,0.554469,1.559682
2957,frozenset(['Organic Unsweetened Almond Milk']),frozenset(['Organic Garlic']),0.109602,0.494087,1.520192
579,frozenset(['Organic Unsweetened Almond Milk']),frozenset(['Organic Baby Spinach']),0.109602,0.644076,1.512823
11895,frozenset(['Organic Unsweetened Almond Milk']),frozenset(['Organic Strawberries']),0.109602,0.523994,1.455737
14356,frozenset(['Organic Unsweetened Almond Milk']),frozenset(['Limes']),0.109602,0.468160,1.290480


In [435]:
print 'recommendation for soy lactosefree products'
[x.print_rows(max_column_width = 300) for x in personal_recommend4[1]]

recommendation for soy lactosefree products
+---------------------------------+-------------------------------------+----------------+------+
|           product_name          |               similar               |     score      | rank |
+---------------------------------+-------------------------------------+----------------+------+
| Organic Unsweetened Almond Milk |     Organic Reduced Fat 2% Milk     | 0.999997019768 |  1   |
| Organic Unsweetened Almond Milk |   Italian Sparkling Mineral Water   | 0.999995946884 |  2   |
| Organic Unsweetened Almond Milk |          Organic Eggs Large         | 0.999995529652 |  3   |
| Organic Unsweetened Almond Milk |        Natural Artesian Water       | 0.999995112419 |  4   |
| Organic Unsweetened Almond Milk |     Organic Original Almond Milk    | 0.999994337559 |  5   |
| Organic Unsweetened Almond Milk |             Organic Milk            | 0.999992966652 |  6   |
| Organic Unsweetened Almond Milk |              Large Eggs             | 

[None, None, None, None, None]

I want to recommend products for customers in group 4 who like soy lactosefree products. I do both association analysis and personalized model analysis. Although association rules model help us to find some rules for us, but we can see lift values are not very high and recommended products are all vegetables or fruits. These rules might not provides us enough information becasue fruits and vegetables are products with very high frequency. Personalized model provides some information for us. Here prints recommended products for 5 top products in soy lactosefree aisle. We can focus on the first one Organic Unsweetened Almond Milk. Recommendated products include reduced fat, sparkling mineral water, eggs tuna salad …… Most recommended products are very healthy and wiht low fat, which are corresponding to characteristics of group 4 custoemrs. So, the recommendation results make sense. 

In [437]:
print 'recommendation for fresh herbs'
[x.print_rows(max_column_width = 300) for x in personal_recommend4[0]]

recommendation for fresh herbs
+------------------+---------------------------------------------+----------------+------+
|   product_name   |                   similar                   |     score      | rank |
+------------------+---------------------------------------------+----------------+------+
| Organic Cilantro |         Organic Kosher Dill Pickles         | 0.999394476414 |  1   |
| Organic Cilantro |           Outrageous B & B Pickles          | 0.999060750008 |  2   |
| Organic Cilantro |  Organic Fruit Yogurt Smoothie Peach Banana | 0.999053239822 |  3   |
| Organic Cilantro |            Organic Hearts Of Palm           | 0.998860239983 |  4   |
| Organic Cilantro |            Classic Round Crackers           | 0.998631834984 |  5   |
| Organic Cilantro |       100% Recycled Bath Tissue Rolls       | 0.998419106007 |  6   |
| Organic Cilantro |       Sunny Days Strawberry Snack Bars      | 0.998270273209 |  7   |
| Organic Cilantro |              Super Greens Salad       

[None, None, None, None, None]